# Module 04 - Notebook 05: Content Moderation API

## Learning Objectives
- Use OpenAI Moderation API
- Filter harmful content automatically
- Implement custom moderation logic
- Handle edge cases and false positives

---

## 1. OpenAI Moderation API Basics

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def moderate_content(text: str) -> dict:
    """Check content with OpenAI Moderation API."""
    
    response = client.moderations.create(input=text)
    result = response.results[0]
    
    return {
        "flagged": result.flagged,
        "categories": result.categories.model_dump(),
        "category_scores": result.category_scores.model_dump()
    }

# Test with safe content
safe_text = "I love learning about artificial intelligence!"
result = moderate_content(safe_text)

print(f"Text: {safe_text}")
print(f"Flagged: {result['flagged']}")
print(f"\nCategories flagged: {[k for k, v in result['categories'].items() if v]}")

## 2. Moderation Categories

The API checks for:
- **hate**: Hateful content
- **hate/threatening**: Hateful with violence
- **harassment**: Harassment
- **harassment/threatening**: Harassment with violence
- **self-harm**: Self-harm content
- **self-harm/intent**: Intent to self-harm
- **self-harm/instructions**: Instructions for self-harm
- **sexual**: Sexual content
- **sexual/minors**: Sexual content involving minors
- **violence**: Violent content
- **violence/graphic**: Graphic violence

In [ ]:
# Test various content types
test_texts = [
    "This is a normal, friendly message.",
    "I want to hurt myself.",  # Self-harm
    "I hate everyone and want to cause violence.",  # Hate + Violence
]

for text in test_texts:
    result = moderate_content(text)
    status = "🚨 FLAGGED" if result["flagged"] else "✓ Safe"
    
    print(f"{status}: {text[:50]}")
    if result["flagged"]:
        flagged_cats = [k for k, v in result["categories"].items() if v]
        print(f"  Categories: {', '.join(flagged_cats)}")
    print()

## 3. Building a Moderated Chatbot

In [ ]:
class ModeratedChatbot:
    """Chatbot with input/output moderation."""
    
    def __init__(self, client: OpenAI):
        self.client = client
    
    def is_safe(self, text: str) -> tuple[bool, dict]:
        """Check if text passes moderation."""
        result = moderate_content(text)
        return not result["flagged"], result
    
    def chat(self, user_input: str) -> str:
        """Chat with input/output moderation."""
        
        # Moderate input
        input_safe, input_result = self.is_safe(user_input)
        if not input_safe:
            flagged = [k for k, v in input_result["categories"].items() if v]
            return f"⚠️ Your message was flagged for: {', '.join(flagged)}. Please rephrase."
        
        # Get response
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful, respectful assistant."},
                {"role": "user", "content": user_input}
            ]
        )
        
        output = response.choices[0].message.content
        
        # Moderate output
        output_safe, output_result = self.is_safe(output)
        if not output_safe:
            return "⚠️ I cannot provide that response. Let me try again."
        
        return output

# Test moderated chatbot
bot = ModeratedChatbot(client)

test_inputs = [
    "What's the weather like?",
    "I hate you and want to hurt you",  # Will be blocked
    "Tell me about machine learning"
]

for inp in test_inputs:
    print(f"User: {inp}")
    print(f"Bot: {bot.chat(inp)}")
    print("\n" + "="*60 + "\n")

## 4. Score-Based Moderation

Use scores for fine-grained control instead of binary flagged/not flagged.

In [ ]:
import pandas as pd

def detailed_moderation(text: str) -> pd.DataFrame:
    """Show detailed category scores."""
    
    result = moderate_content(text)
    scores = result["category_scores"]
    
    # Create DataFrame
    df = pd.DataFrame([
        {"Category": k, "Score": f"{v:.4f}", "Flagged": result["categories"][k]}
        for k, v in scores.items()
    ])
    
    return df.sort_values("Score", ascending=False)

# Analyze borderline content
text = "I'm really frustrated with this situation."
print(f"Analyzing: {text}\n")
print(detailed_moderation(text))

## 5. Custom Thresholds

In [ ]:
class CustomModeration:
    """Moderation with custom thresholds."""
    
    def __init__(self, thresholds: dict = None):
        # Default: More strict for violence, less strict for others
        self.thresholds = thresholds or {
            "violence": 0.3,  # Lower = more strict
            "hate": 0.5,
            "harassment": 0.5,
            "self-harm": 0.4,
            "sexual": 0.6,
        }
    
    def moderate(self, text: str) -> dict:
        """Apply custom thresholds."""
        result = moderate_content(text)
        scores = result["category_scores"]
        
        violations = []
        for category, threshold in self.thresholds.items():
            # Check all related categories
            for cat_name, score in scores.items():
                if category in cat_name and score > threshold:
                    violations.append((cat_name, score))
        
        return {
            "blocked": len(violations) > 0,
            "violations": violations,
            "original_flagged": result["flagged"]
        }

# Test custom moderation
moderator = CustomModeration()

test = "I'm going to destroy my computer!"
result = moderator.moderate(test)

print(f"Text: {test}")
print(f"Custom blocked: {result['blocked']}")
print(f"OpenAI flagged: {result['original_flagged']}")
print(f"Violations: {result['violations']}")

## 6. Batch Moderation

In [ ]:
def batch_moderate(texts: list[str]) -> pd.DataFrame:
    """Moderate multiple texts efficiently."""
    
    results = []
    
    for text in texts:
        mod_result = moderate_content(text)
        results.append({
            "text": text[:50] + "..." if len(text) > 50 else text,
            "flagged": mod_result["flagged"],
            "categories": ", ".join([k for k, v in mod_result["categories"].items() if v]) or "None"
        })
    
    return pd.DataFrame(results)

# Test batch moderation
comments = [
    "This is a great product!",
    "I hate this so much",
    "The quality is terrible",
    "Amazing experience, highly recommend",
    "You're all idiots"
]

print("Batch Moderation Results:\n")
print(batch_moderate(comments))

## 7. Handling False Positives

In [ ]:
def smart_moderation(text: str, context: str = None) -> dict:
    """Moderation with context awareness."""
    
    result = moderate_content(text)
    
    # Add context-based overrides
    if context == "medical" and result["categories"].get("self-harm", False):
        # Medical discussions might trigger false positives
        if "treatment" in text.lower() or "therapy" in text.lower():
            result["flagged"] = False
            result["override_reason"] = "Medical context"
    
    return result

# Test context-aware moderation
medical_text = "I'm learning about self-harm prevention and treatment options."

print("Without context:")
print(moderate_content(medical_text)["flagged"])

print("\nWith medical context:")
print(smart_moderation(medical_text, context="medical")["flagged"])

## Exercise: Build a Comment Moderation System

Create a system that:
1. Accepts user comments
2. Moderates using OpenAI API
3. Logs flagged content
4. Provides user-friendly feedback
5. Allows appeals for false positives

In [ ]:
# TODO: Complete this exercise
class CommentModerationSystem:
    """Full-featured comment moderation."""
    
    def __init__(self):
        self.flagged_log = []
    
    def submit_comment(self, user_id: str, comment: str) -> dict:
        """Submit and moderate a comment."""
        # Your code here
        pass
    
    def appeal(self, comment_id: str, reason: str) -> bool:
        """Appeal a moderation decision."""
        # Your code here
        pass

# Test your system
# system = CommentModerationSystem()
# system.submit_comment("user123", "This is inappropriate content")

## Summary

You learned:
- ✅ Using OpenAI Moderation API
- ✅ Understanding moderation categories
- ✅ Building moderated chatbots
- ✅ Custom thresholds and batch processing
- ✅ Handling false positives

## Best Practices
1. **Moderate both input and output**
2. **Use scores for nuanced decisions**
3. **Provide clear user feedback**
4. **Log flagged content for review**
5. **Allow appeals for false positives**
6. **Consider context when possible**

## Next Steps
- 📘 Notebook 06: Bias Testing